# 📊 Modelo 3: K-Nearest Neighbors (KNN)
## Clasificación por Vecinos Más Cercanos

---

## 🎯 Objetivo
Clasificar exportaciones colombianas en categorías de valor usando el algoritmo K-Nearest Neighbors (KNN).

El KNN es un algoritmo de aprendizaje supervisado que clasifica nuevos ejemplos basándose en la similitud con los k ejemplos más cercanos en el espacio de características.

## 📊 Variables
- **Target (Variable objetivo)**: `Categoria_Valor` - Clasificación del valor FOB en categorías (Bajo/Medio/Alto/Muy Alto)
- **Features (Características)**: Variables numéricas y categóricas codificadas relacionadas con las exportaciones

## 📋 Contenido
1. Importación de librerías
2. Carga y exploración de datos
3. Preprocesamiento de datos
4. Preparación de features y target
5. Entrenamiento del modelo KNN
6. Evaluación del modelo
7. Validación cruzada
8. Guardado del modelo

## 1. Importación de Librerías

Importamos todas las librerías necesarias para el análisis y modelado.

In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Librerías de machine learning
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
import pickle

# Configuración de gráficos
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print('✅ Librerías importadas correctamente')

## 2. Carga de Datos

Cargamos el dataset original desde el archivo Excel y realizamos una exploración inicial.

In [ ]:
# Cargar el dataset desde Excel
df = pd.read_excel('../DATA/DATAPROYECTO.xlsx', sheet_name='Detalle')

print(f'📦 Dataset cargado exitosamente')
print(f'   Dimensiones: {df.shape[0]:,} filas × {df.shape[1]} columnas')
print(f'   Tamaño en memoria: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB')

# Vista rápida de las primeras filas
df.head()

## 3. Preprocesamiento de Datos

Realizamos el preprocesamiento necesario para preparar los datos para el modelo KNN.

In [ ]:
# Crear una copia para trabajar
df_processed = df.copy()

print('🔧 Iniciando preprocesamiento...')

# 1. Manejo de valores faltantes
print('\n1️⃣ Tratamiento de valores faltantes:')

# Para columnas numéricas: imputar con la mediana
numeric_cols = df_processed.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    if df_processed[col].isnull().sum() > 0:
        median_val = df_processed[col].median()
        df_processed[col].fillna(median_val, inplace=True)
        print(f'   ✓ {col}: Imputado con mediana = {median_val:.2f}')

# Para columnas categóricas: imputar con 'Desconocido'
categorical_cols = df_processed.select_dtypes(include=['object']).columns
for col in categorical_cols:
    if df_processed[col].isnull().sum() > 0:
        df_processed[col].fillna('Desconocido', inplace=True)
        print(f'   ✓ {col}: Imputado con "Desconocido"')

print(f'\n   Valores nulos restantes: {df_processed.isnull().sum().sum()}')

In [ ]:
# 2. Feature Engineering - Crear variables derivadas
print('\n2️⃣ Feature Engineering:')

# Ratio Peso Bruto/Neto
df_processed['Ratio_Peso_Bruto_Neto'] = df_processed['Peso en kilos brutos'] / (df_processed['Peso en kilos netos'] + 1e-10)
print('   ✓ Ratio_Peso_Bruto_Neto creado')

# Valor por kilogramo
df_processed['Valor_Por_Kg'] = df_processed['Valor FOB (USD)'] / (df_processed['Peso en kilos netos'] + 1e-10)
print('   ✓ Valor_Por_Kg creado')

# Clasificación de valor de exportación (target)
def clasificar_valor(valor):
    if valor < 1000:
        return 'Bajo'
    elif valor < 10000:
        return 'Medio'
    elif valor < 100000:
        return 'Alto'
    else:
        return 'Muy Alto'

df_processed['Categoria_Valor'] = df_processed['Valor FOB (USD)'].apply(clasificar_valor)
print('   ✓ Categoria_Valor creado (Bajo/Medio/Alto/Muy Alto)')

# Ver distribución de la variable target
print('\n📊 Distribución de la variable target:')
print(df_processed['Categoria_Valor'].value_counts())

In [ ]:
# 3. Codificación de variables categóricas
print('\n3️⃣ Codificación de variables categóricas:')

# Variables categóricas a codificar
categorical_to_encode = ['País de Destino', 'Continente Destino', 'Departamento Origen', 'Vía de transporte']

label_encoders = {}

for col in categorical_to_encode:
    if col in df_processed.columns:
        le = LabelEncoder()
        df_processed[col + '_encoded'] = le.fit_transform(df_processed[col].astype(str))
        label_encoders[col] = le
        print(f'   ✓ {col}: Codificado ({df_processed[col].nunique()} categorías → {len(le.classes_)} clases)')

print(f'\n   Total de variables codificadas: {len(label_encoders)}')

## 4. Preparación de Features y Target

Seleccionamos las características (features) y la variable objetivo (target) para el modelo.

In [ ]:
# Definir target y features
target = 'Categoria_Valor'

# Seleccionar features numéricas y codificadas
features = [
    'Peso en kilos netos', 'Peso en kilos brutos', 'Cantidad(es)',
    'Número de artículos', 'Precio Unitario FOB (USD) Peso Neto',
    'País de Destino_encoded', 'Continente Destino_encoded',
    'Departamento Origen_encoded', 'Vía de transporte_encoded',
    'Ratio_Peso_Bruto_Neto', 'Valor_Por_Kg'
]

# Crear dataset final
df_model = df_processed[features + [target]].copy().dropna()
df_model = df_model.replace([np.inf, -np.inf], np.nan).dropna()

# Codificar el target
le_target = LabelEncoder()
y = le_target.fit_transform(df_model[target])

X = df_model[features]

print(f'📊 Dataset preparado:')
print(f'   • Features: {len(features)} variables')
print(f'   • Muestras: {X.shape[0]:,}')
print(f'   • Target: {len(np.unique(y))} clases ({le_target.classes_})')

# Ver balance de clases
plt.figure(figsize=(8, 5))
sns.countplot(x=le_target.inverse_transform(y))
plt.title('Distribución de Clases - Variable Target')
plt.xlabel('Categoría de Valor')
plt.ylabel('Frecuencia')
plt.xticks(rotation=45)
plt.show()

## 5. División y Escalamiento de Datos

Dividimos los datos en conjuntos de entrenamiento y prueba, y escalamos las características.

In [ ]:
# División train/test con estratificación
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f'✂️ División de datos:')
print(f'   • Train: {X_train.shape[0]:,} muestras')
print(f'   • Test: {X_test.shape[0]:,} muestras')

# Escalamiento de features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f'📏 Escalamiento aplicado (StandardScaler)')
print(f'   • Media ≈ 0, Desviación estándar ≈ 1')

## 6. Entrenamiento del Modelo KNN

Entrenamos el modelo K-Nearest Neighbors con los hiperparámetros seleccionados.

In [ ]:
print('🤖 Entrenando modelo K-Nearest Neighbors...')

# Configuración del modelo
model = KNeighborsClassifier(
    n_neighbors=5,           # Número de vecinos
    weights='distance',      # Peso por distancia (más cercano = más peso)
    metric='euclidean'       # Distancia euclidiana
)

# Entrenamiento
model.fit(X_train_scaled, y_train)

print('✅ Modelo entrenado exitosamente')
print(f'   • Algoritmo: K-Nearest Neighbors')
print(f'   • k: {model.n_neighbors}')
print(f'   • Peso: {model.weights}')
print(f'   • Métrica: {model.metric}')

# Predicciones
y_train_pred = model.predict(X_train_scaled)
y_test_pred = model.predict(X_test_scaled)

print('\n🔮 Predicciones realizadas')

## 7. Evaluación del Modelo

Evaluamos el rendimiento del modelo usando varias métricas.

In [ ]:
# Cálculo de métricas
acc_train = accuracy_score(y_train, y_train_pred)
acc_test = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred, average='weighted')
recall = recall_score(y_test, y_test_pred, average='weighted')
f1 = f1_score(y_test, y_test_pred, average='weighted')

print('📊 MÉTRICAS DE EVALUACIÓN:')
print('='*50)
print(f'  Accuracy Train: {acc_train:.4f}')
print(f'  Accuracy Test:  {acc_test:.4f}')
print(f'  Precision:      {precision:.4f}')
print(f'  Recall:         {recall:.4f}')
print(f'  F1-Score:       {f1:.4f}')

# Verificar overfitting/underfitting
if acc_train - acc_test > 0.1:
    print('\n⚠️  Posible overfitting detectado')
elif acc_test < 0.6:
    print('\n⚠️  Posible underfitting detectado')
else:
    print('\n✅ Rendimiento equilibrado')

In [ ]:
# Reporte de clasificación detallado
print('\n📋 REPORTE DE CLASIFICACIÓN DETALLADO:')
print('='*50)
print(classification_report(y_test, y_test_pred, target_names=le_target.classes_))

## 8. Matriz de Confusión

Visualizamos la matriz de confusión para entender mejor los errores de clasificación.

In [ ]:
# Matriz de confusión
cm = confusion_matrix(y_test, y_test_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=le_target.classes_, yticklabels=le_target.classes_)
plt.title('Matriz de Confusión - Modelo KNN', fontweight='bold', fontsize=14)
plt.xlabel('Predicción', fontsize=12)
plt.ylabel('Real', fontsize=12)
plt.tight_layout()
plt.show()

print('📊 Interpretación de la matriz de confusión:')
print('   • Diagonal principal: Predicciones correctas')
print('   • Fuera de diagonal: Errores de clasificación')

## 9. Validación Cruzada

Realizamos validación cruzada para obtener una estimación más robusta del rendimiento.

In [ ]:
# Validación cruzada
cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='accuracy')

print('🔄 VALIDACIÓN CRUZADA (5-fold):')
print('='*50)
print(f'  Scores individuales: {cv_scores}')
print(f'  Accuracy promedio: {cv_scores.mean():.4f}')
print(f'  Desviación estándar: {cv_scores.std():.4f}')
print(f'  Rango: [{cv_scores.min():.4f}, {cv_scores.max():.4f}]')

# Comparación con accuracy de test
print(f'\n📊 Comparación:')
print(f'  CV Accuracy: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}')
print(f'  Test Accuracy: {acc_test:.4f}')

if abs(cv_scores.mean() - acc_test) < 0.05:
    print('✅ Resultados consistentes')
else:
    print('⚠️  Diferencia significativa entre CV y test')

## 10. Guardado del Modelo

Guardamos el modelo entrenado junto con los objetos necesarios para su uso posterior.

In [ ]:
# Preparar paquete del modelo
model_package = {
    'model': model,
    'scaler': scaler,
    'label_encoder_target': le_target,
    'label_encoders_features': label_encoders,
    'features': features,
    'target': target,
    'metrics': {
        'accuracy_train': acc_train,
        'accuracy_test': acc_test,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'cv_accuracy_mean': cv_scores.mean(),
        'cv_accuracy_std': cv_scores.std()
    },
    'model_info': {
        'algorithm': 'K-Nearest Neighbors',
        'n_neighbors': model.n_neighbors,
        'weights': model.weights,
        'metric': model.metric,
        'n_features': len(features),
        'n_classes': len(le_target.classes_),
        'classes': list(le_target.classes_)
    }
}

# Guardar el modelo
with open('model_knn.pkl', 'wb') as f:
    pickle.dump(model_package, f)

print('💾 Modelo guardado exitosamente')
print('   • Archivo: model_knn.pkl')
print('   • Contiene: modelo, scaler, encoders y métricas')

# Verificar que se guardó correctamente
import os
if os.path.exists('model_knn.pkl'):
    size = os.path.getsize('model_knn.pkl') / 1024
    print(f'   • Tamaño: {size:.2f} KB')
    print('✅ Verificación exitosa')
else:
    print('❌ Error al guardar el modelo')

## 🎯 Conclusiones

### Resumen del Modelo KNN:

**Fortalezas:**
- Algoritmo simple e interpretable
- No requiere supuestos sobre la distribución de los datos
- Funciona bien con datasets pequeños a medianos
- Robusto a outliers cuando se usa distancia apropiada

**Limitaciones:**
- Sensible a la escala de las variables (requiere escalamiento)
- Computacionalmente costoso en datasets grandes
- Sensible al número k y a la métrica de distancia
- No maneja bien datos de alta dimensionalidad

### Resultados Obtenidos:
- **Accuracy en test:** {acc_test:.4f}
- **F1-Score:** {f1:.4f}
- **Validación cruzada:** {cv_scores.mean():.4f} ± {cv_scores.std():.4f}

### Recomendaciones:
1. **Optimización de hiperparámetros:** Usar GridSearchCV para encontrar el mejor valor de k
2. **Selección de features:** Considerar reducción de dimensionalidad si es necesario
3. **Balanceo de clases:** Si las clases están desbalanceadas, considerar técnicas de oversampling/undersampling
4. **Comparación con otros modelos:** Evaluar contra otros algoritmos de clasificación

---

**📊 Modelo KNN completado exitosamente** ✅

*Los datos están listos para ser utilizados en la aplicación o para comparaciones con otros modelos.*